# Classify snow-covered area (SCA) in Landsat, Sentinel-2, and MODIS surface reflectance iamgery: full pipeline

Rainey Aberle

Department of Geosciences, Boise State University

2022

### Requirements:
- Area of Interest (AOI) shapefile: where snow will be classified in each image. 
- Google Earth Engine (GEE) account: used to pull DEM over the AOI. Sign up for a free account [here](). 

### Outline:
__0. Setup__ paths in directory, AOI file location - _modify this section!_

__1. Load images__ over the AOI since 2016. 

__2. Classify SCA__ and use the snow elevations distribution to estimate the seasonal snowline

-------


### 0. Setup

#### Define paths in directory and desired settings. 
Modify lines located within the following:

`#### MODIFY HERE ####`  

`#####################`

In [ ]:
##### MODIFY HERE #####
# -----Path to planet-snow
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/planet-snow/'

# -----Paths in directory
site_name = 'Wolverine'
# path to AOI including the name of the shapefile
AOI_fn = base_path+'../../GIS_data/RGI_outlines/'+site_name+'_RGI.shp'
# path for output figures
figures_out_path = base_path+'../study-sites/'+site_name+'/figures/SCA_Landsat/'

# -----Define maximum cloud cover filter for image search
cloud_cover_max = 10

# -----Determine settings
plot_results = True # = True to plot figures of results for each image where applicable
crop_to_AOI = True # = True to crop images to AOI before calculating SCA
save_outputs = True # = True to save SCA images to file
save_figures = True # = True to save SCA output figures to file

#######################

# -----Import packages
import xarray as xr
import wxee as wx
import os
import numpy as np
import glob
from osgeo import gdal
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from matplotlib.patches import Rectangle
from matplotlib import pyplot as plt, dates
import rasterio as rio
import rasterio.features
from rasterio.mask import mask
from rasterio.plot import show
from shapely.geometry import Polygon, shape
import shapely.geometry
from scipy.interpolate import interp2d
from scipy import stats
import pandas as pd
import geopandas as gpd
import geemap
import math
import sys
import ee
import fiona
import pickle

# -----Add path to functions
sys.path.insert(1, base_path+'functions/')
import ps_pipeline_utils as f

# -----Load AOI as geopandas.GeoDataFrame
AOI = gpd.read_file(AOI_fn)

#### Authenticate and initialize Google Earth Engine (GEE). 

__Note:__ The first time you run the following cell, you will be asked to authenticate your GEE account for use in this notebook. This will send you to an external web page, where you will walk through the GEE authentication workflow and copy an authentication code back in this notebook when prompted. 

In [ ]:
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

### 1. Load images over the AOI since 2016


In [ ]:
# -----Reformat AOI for clipping images
# reproject AOI to WGS 84 for compatibility with images
AOI_WGS = AOI.to_crs(4326)
# reformat AOI_WGS bounding box as ee.Geometry for clipping DEM
AOI_WGS_bb_ee = ee.Geometry.Polygon(
                        [[[AOI_WGS.geometry.bounds.minx[0], AOI_WGS.geometry.bounds.miny[0]],
                          [AOI_WGS.geometry.bounds.maxx[0], AOI_WGS.geometry.bounds.miny[0]],
                          [AOI_WGS.geometry.bounds.maxx[0], AOI_WGS.geometry.bounds.maxy[0]],
                          [AOI_WGS.geometry.bounds.minx[0], AOI_WGS.geometry.bounds.maxy[0]],
                          [AOI_WGS.geometry.bounds.minx[0], AOI_WGS.geometry.bounds.miny[0]]]
                        ])

# -----Define start and end dates and months
date_range_start = '2022-01-01'
date_range_end = '2022-12-01'
month_start = 5
month_end = 10

# -----Query GEE for Landsat, Sentinel, and MODIS images
L_col = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
         .filter(ee.Filter.lt("CLOUD_COVER", cloud_cover_max))
         .filterDate(ee.Date(date_range_start), ee.Date(date_range_end))
         .filter(ee.Filter.calendarRange(month_start, month_end, 'month'))
         .filterBounds(AOI_WGS_bb_ee))
S_col = (ee.ImageCollection("COPERNICUS/S2_SR")
         .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover_max))
         .filterDate(ee.Date(date_range_start), ee.Date(date_range_end))
         .filter(ee.Filter.calendarRange(month_start, month_end, 'month'))
         .filterBounds(AOI_WGS_bb_ee))
M_col = (ee.ImageCollection("MODIS/061/MOD09GA").merge(ee.ImageCollection("MODIS/061/MYD09GA"))
         .filterDate(ee.Date(date_range_start), ee.Date(date_range_end))
         .filter(ee.Filter.calendarRange(month_start, month_end, 'month'))
         .filterBounds(AOI_WGS_bb_ee))

# -----Clip images to AOI
def clip_image(im):
    return im.clip(AOI_WGS_bb_ee)
L_col_clipped = L_col.map(clip_image)
S_col_clipped = S_col.map(clip_image)
M_col_clipped = M_col.map(clip_image)

In [ ]:
# -----Determine optimal UTM zone EPSG code
epsg_code = f.convert_wgs_to_utm((AOI_WGS.geometry.bounds.maxx[0] - AOI_WGS.geometry.bounds.minx[0]) + AOI_WGS.geometry.bounds.minx[0],
                              (AOI_WGS.geometry.bounds.maxy[0] - AOI_WGS.geometry.bounds.miny[0]) + AOI_WGS.geometry.bounds.miny[0])

# -----Convert image collections to Xarray.Datasets
L_col_xr = L_col_clipped.wx.to_xarray(crs='EPSG:'+epsg_code, scale=30)
S_col_xr = S_col_clipped.wx.to_xarray(crs='EPSG:'+epsg_code, scale=20)
M_col_xr = M_col_clipped.wx.to_xarray(crs='EPSG:'+epsg_code, scale=500)